In [ ]:
# Reference: 
# OPQ demo: https://github.com/facebookresearch/faiss/issues/411
# Load preprocessing operator: https://github.com/facebookresearch/faiss/blob/master/benchs/bench_gpu_1bn.py

In [1]:
from __future__ import print_function
import os
import sys
import time
import numpy as np
import re
import faiss
import inspect
from multiprocessing.dummy import Pool as ThreadPool
from matplotlib import pyplot

In [24]:
# create an inverted index
nlist = 1024 # IVF cell number
m = 16 # PQ partitions
kbits = 8 # 2^5 = 32, 2^8 = 256
d = 128 # dim of db vectors
n = 100000 # num of db vectors

In [25]:
# fill it in
xb = faiss.rand((n, d), 1234)
# index.train(xb)
# index.add(xb)

In [26]:
# coarse_quantizer = faiss.IndexFlatL2(d)
# index = faiss.IndexIVFPQ(coarse_quantizer, d, nlist, m, kbits)

In [27]:
opq = faiss.OPQMatrix(d, m)
opq.train(xb)
xt = opq.apply_py(xb)
pq = faiss.ProductQuantizer(d, m, kbits)
pq.train(xt)
codes = pq.compute_codes(xb)

In [37]:
print("opq.A", opq.A)
print("opq.M", opq.M)
vec = faiss.vector_to_array(opq.A)
print(vec.shape) # 128 * 128
print(vec)

opq.A <faiss.swigfaiss_avx2.FloatVector; proxy of <Swig Object of type 'std::vector< float > *' at 0x7fd4b6dc34b0> >
opq.M 16
(16384,)
[-0.09090281  0.09487255  0.09818375 ...  0.15406719 -0.08668799
  0.02816606]


In [7]:
codes

array([[139,  11, 146, ..., 209, 149,  66],
       [227,  11, 146, ..., 153, 247,  44],
       [139,  11,  42, ...,  74, 149,  44],
       ...,
       [154,  11,  92, ...,  74,   8, 122],
       [139,  11,  42, ...,  40, 247,  44],
       [227,  11,  42, ...,  40, 149,  44]], dtype=uint8)

array([[-0.09090281,  0.09487255,  0.09818375, ..., -0.03984596,
        -0.08296149, -0.00214473],
       [ 0.08088752, -0.10553612, -0.03300218, ...,  0.05117955,
        -0.13224462,  0.1514061 ],
       [ 0.09209212,  0.10065087, -0.03060819, ..., -0.06132574,
        -0.14832038, -0.18674988],
       ...,
       [ 0.00442195,  0.11818414, -0.00067089, ...,  0.01547248,
        -0.04173023,  0.05076025],
       [-0.09678233,  0.18215787,  0.06500293, ...,  0.00624335,
         0.01511531,  0.11763625],
       [-0.0023328 , -0.06425092,  0.07417437, ...,  0.15406719,
        -0.08668799,  0.02816606]], dtype=float32)

In [44]:
OPQ_mat = np.reshape(vec, (128,128))
x_verify = np.matmul(xb, OPQ_mat)
print(x_verify)

OPQ_mat_T = np.transpose(OPQ_mat)
x_verify_T = np.matmul(xb, OPQ_mat_T)
print(x_verify_T)

[[ 0.33077395  0.80827886 -0.7675641  ... -0.03774748 -0.6464313
   0.46730372]
 [-0.2964133   0.38181987 -0.5297576  ...  0.1065072  -0.47322565
   0.23144089]
 [-0.4822103   0.94808215 -0.4776723  ...  0.46175957 -0.605573
   0.4937843 ]
 ...
 [-0.19966108  0.25764626 -0.27646154 ...  0.22020607 -0.75905144
   0.4758157 ]
 [-0.32742465  0.21110094 -0.7843784  ...  0.02842903 -0.315985
   0.15336634]
 [-0.48186213  0.23942779  0.4081095  ...  0.61109334 -0.38750818
   0.07828852]]
[[ 0.66452736 -0.00846681 -0.56798387 ...  0.9972061   0.9702721
   0.04158473]
 [ 0.31273285 -0.13870661 -0.01504563 ...  0.2038115   1.565992
   0.07616587]
 [ 1.1000265  -0.16824633 -1.1169357  ...  0.98503643  1.737572
   0.26097906]
 ...
 [ 0.7410809  -0.3422086  -0.2699805  ...  0.66872954  1.7959784
   0.59956545]
 [ 1.0574831  -0.29653358 -0.24074297 ...  1.1340361   1.2729566
  -0.14015266]
 [ 0.573179    0.3359762  -0.14394149 ...  1.0408187   1.7643989
   0.2095789 ]]


In [43]:
print(xt, xt.shape)

[[ 0.66452736 -0.00846681 -0.56798387 ...  0.9972061   0.9702721
   0.04158473]
 [ 0.31273285 -0.13870661 -0.01504563 ...  0.2038115   1.565992
   0.07616587]
 [ 1.1000265  -0.16824633 -1.1169357  ...  0.98503643  1.737572
   0.26097906]
 ...
 [ 0.7410809  -0.3422086  -0.2699805  ...  0.66872954  1.7959784
   0.59956545]
 [ 1.0574831  -0.29653358 -0.24074297 ...  1.1340361   1.2729566
  -0.14015266]
 [ 0.573179    0.3359762  -0.14394149 ...  1.0408187   1.7643989
   0.2095789 ]] (100000, 128)


In [45]:
np.allclose(x_verify_T, xt)
# The transposed version of OPQ Matrix is used to preprocessing input vector

True

In [54]:
preproc = faiss.read_VectorTransform(
    "../trained_GPU_indexes/bench_gpu_SIFT100M_OPQ16,IVF16384,PQ16/preproc_SIFT100M_OPQ16.vectrans")
loaded_OPQ_mat = faiss.vector_to_array(preproc.A)
print(loaded_OPQ_mat, loaded_OPQ_mat.shape)

[ 0.04449736  0.14647843 -0.00979544 ...  0.07864936 -0.02518905
 -0.11654815] (16384,)


In [53]:
inspect.getmembers(preproc)

[('A',
  <faiss.swigfaiss_avx2.FloatVector; proxy of <Swig Object of type 'std::vector< float > *' at 0x7fd4b6c92d20> >),
 ('__class__', faiss.swigfaiss_avx2.LinearTransform),
 ('__del__',
  <bound method LinearTransform.<lambda> of <faiss.swigfaiss_avx2.LinearTransform; proxy of <Swig Object of type 'faiss::LinearTransform *' at 0x7fd4eb4a8f00> >>),
 ('__delattr__',
  <method-wrapper '__delattr__' of LinearTransform object at 0x7fd4b6c598d0>),
 ('__dict__',
  {'this': <Swig Object of type 'faiss::LinearTransform *' at 0x7fd4eb4a8f00>}),
 ('__dir__', <function LinearTransform.__dir__>),
 ('__doc__', None),
 ('__eq__',
  <method-wrapper '__eq__' of LinearTransform object at 0x7fd4b6c598d0>),
 ('__format__', <function LinearTransform.__format__>),
 ('__ge__',
  <method-wrapper '__ge__' of LinearTransform object at 0x7fd4b6c598d0>),
 ('__getattr__',
  <bound method LinearTransform.<lambda> of <faiss.swigfaiss_avx2.LinearTransform; proxy of <Swig Object of type 'faiss::LinearTransform *' a

In [46]:
inspect.getmembers(faiss)

[('ArrayInvertedLists', faiss.swigfaiss_avx2.ArrayInvertedLists),
 ('ArrayInvertedLists_swigregister',
  <function faiss._swigfaiss_avx2.ArrayInvertedLists_swigregister>),
 ('AutoTuneCriterion', faiss.swigfaiss_avx2.AutoTuneCriterion),
 ('AutoTuneCriterion_swigregister',
  <function faiss._swigfaiss_avx2.AutoTuneCriterion_swigregister>),
 ('BitstringReader', faiss.swigfaiss_avx2.BitstringReader),
 ('BitstringReader_swigregister',
  <function faiss._swigfaiss_avx2.BitstringReader_swigregister>),
 ('BitstringWriter', faiss.swigfaiss_avx2.BitstringWriter),
 ('BitstringWriter_swigregister',
  <function faiss._swigfaiss_avx2.BitstringWriter_swigregister>),
 ('BufferList', faiss.swigfaiss_avx2.BufferList),
 ('BufferList_swigregister',
  <function faiss._swigfaiss_avx2.BufferList_swigregister>),
 ('BufferedIOReader', faiss.swigfaiss_avx2.BufferedIOReader),
 ('BufferedIOReader_swigregister',
  <function faiss._swigfaiss_avx2.BufferedIOReader_swigregister>),
 ('BufferedIOWriter', faiss.swigfais